In [2]:
import pandas as pd
import json 
import os

In [3]:
id_maxs = []
maxs = [] 
for file in os.listdir("./json_data"): 
    df = pd.read_json(os.path.join("./json_data", file), encoding="utf-8")
    content = df["content"]
    content = content.apply(func=lambda x: len(x))
    id_maxs.append(content.idxmax())
    maxs.append(content.max())

In [3]:
df = pd.read_json("./json_data/fb_comment_text_0.json", encoding="utf-8")

In [23]:
df["content"].loc[21]

'-Bạn nào cần vay tiền gấp bấm vào link bên dưới rồi điền thông tin nha\n-Hoặc ib mình tư vấn\n -khoảng vay từ 10tr-80tr \n-thủ tục vay đơn giản\n\n👉link : https://shorten.asia/8VFGxTbN'

In [4]:
print(max(maxs))

11449
